In [1]:
%pip install -r requirements.txt

Obtaining file:///D:/Documents/blockchain/GenAi/Project/AskPaktour (from -r requirements.txt (line 7))
Note: you may need to restart the kernel to use updated packages.


ERROR: file:///D:/Documents/blockchain/GenAi/Project/AskPaktour (from -r requirements.txt (line 7)) does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [3]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [6]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [7]:
extracted_data = load_pdf("data/")

Error loading file data\Pak_Tourism_Guide-Sept_2021-merged.pdf


ImportError: pypdf package not found, please install it with `pip install pypdf`

In [5]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks




In [6]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [7]:
text_chunks

[Document(page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION', metadata={'source': 'data\\Medical_book.pdf', 'page': 1}),
 Document(page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B1', metadata={'source': 'data\\Medical_book.pdf', 'page': 2}),
 Document(page_content='STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor\nChristine B. Jeryan, Managing Editor\nDonna Olendorf, Senior Editor\nStacey Blachford, Associate Editor\nKate Kretschmann, Melissa C. McDade, Ryan\nThomason, Assistant Editors\nMark Springer, Technical Specialist\nAndrea Lopeman, Programmer/Analyst\nBarbara J. Yarrow, Manager, Imaging and Multimedia\nContent\nRobyn V . Young, Project Manager, Imaging and\nMultimedia Content\nDean Dauphinais, Senior Editor, Imaging and', metadata={'source': 'data\\Medical_book.pdf', 'page': 3}),
 Document(page_content='Multimedia Content\nKelly A.

In [8]:
import os
from langchain.llms import OpenAI
from langchain_objectbox.vectorstores import ObjectBox ##vector Database
from langchain_openai import OpenAIEmbeddings
os.environ["OPENAI_API_KEY"]=""


In [9]:
from langchain_openai import OpenAIEmbeddings
vectors = ObjectBox.from_documents(text_chunks, OpenAIEmbeddings(), embedding_dimensions=768)
vectors

In [10]:
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI

In [11]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=vectors.as_retriever(),
                                  return_source_documents=True)

In [12]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [13]:
# full example
query = "What are Allergies?"

In [14]:
llm_response = qa_chain.invoke(query)


In [15]:
llm_response

{'query': 'What are Allergies?',
 'result': ' Allergies are abnormal reactions of the immune system that occur in response to otherwise harmless substances.',
 'source_documents': [Document(page_content='Allergy and Immunology. Boston: Little, Brown and Co.,\n1995.\nNovick, N. L. You Can Do Something About Your Allergies.\nNew York: Macmillan, 1994.\nWeil, A. Natural Health, Natural Medicine: A Comprehensive\nManual for Wellness and Self-Care. New York: Houghton\nMifflin, 1995.\nRichard Robinson\nAllergies\nDefinition\nAllergies are abnormal reactions of the immune sys-\ntem that occur in response to otherwise harmless sub-stances.', metadata={'page': 127, 'source': 'data\\Medical_book.pdf'}),
  Document(page_content='Description\nAllergies are among the most common of medical\ndisorders. It is estimated that 60 million Americans, ormore than one in every five people, suffer from someform of allergy, with similar proportions throughoutmuch of the rest of the world. Allergy is the singl

In [16]:
llm_response

{'query': 'What are Allergies?',
 'result': ' Allergies are abnormal reactions of the immune system that occur in response to otherwise harmless substances.',
 'source_documents': [Document(page_content='Allergy and Immunology. Boston: Little, Brown and Co.,\n1995.\nNovick, N. L. You Can Do Something About Your Allergies.\nNew York: Macmillan, 1994.\nWeil, A. Natural Health, Natural Medicine: A Comprehensive\nManual for Wellness and Self-Care. New York: Houghton\nMifflin, 1995.\nRichard Robinson\nAllergies\nDefinition\nAllergies are abnormal reactions of the immune sys-\ntem that occur in response to otherwise harmless sub-stances.', metadata={'page': 127, 'source': 'data\\Medical_book.pdf'}),
  Document(page_content='Description\nAllergies are among the most common of medical\ndisorders. It is estimated that 60 million Americans, ormore than one in every five people, suffer from someform of allergy, with similar proportions throughoutmuch of the rest of the world. Allergy is the singl

In [31]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [32]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [33]:
while True:
    user_input = input("Input Prompt: ")
    if user_input.lower() == "exit":  # Exit condition
        print("Exiting the loop.")
        break  # Breaks out of the while loop
    result = qa_chain({"query": user_input})
    print("Response: ", result["result"])


Response:   Acne is a skin condition characterized by raised bumps, pimples, and cysts that form on the face, neck, shoulders, and upper back. It is caused by inflammation of the sebaceous glands and is the most common skin disease, affecting nearly 17 million people in the United States. It typically begins at puberty and worsens during adolescence, with up to 20% of women developing mild acne. 
Exiting the loop.
